In [1]:
from pyrootutils import setup_root
PROJECT_ROOT = setup_root(".", indicator=".gitignore", pythonpath=True)

import torch
from core import (create_dataloaders, create_model, Trainer, Evaluator,
                  plot_training_history, print_training_summary)
                  
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [2]:
# 사용 가능한 모델 목록 확인 (선택 사항)
from core import print_available_models

print_available_models()  



Available Models for Binary Classification

RESNET:
  1. resnet18
  2. resnet34
  3. resnet50
  4. resnet101
  5. resnet152

EFFICIENTNET:
  1. efficientnet_b0
  2. efficientnet_b1
  3. efficientnet_b2
  4. efficientnet_b3
  5. efficientnet_b4
  6. efficientnet_b5
  7. efficientnet_b6
  8. efficientnet_b7

EFFICIENTNET_V2:
  1. efficientnet_v2_s
  2. efficientnet_v2_m
  3. efficientnet_v2_l

CONVNEXT:
  1. convnext_tiny
  2. convnext_small
  3. convnext_base
  4. convnext_large

DENSENET:
  1. densenet121
  2. densenet161
  3. densenet169
  4. densenet201

VIT:
  1. vit_b_16
  2. vit_b_32
  3. vit_l_16
  4. vit_l_32

SWIN:
  1. swin_t
  2. swin_s
  3. swin_b

SWIN_V2:
  1. swin_v2_t
  2. swin_v2_s
  3. swin_v2_b

REGNET:
  1. regnet_y_400mf
  2. regnet_y_800mf
  3. regnet_y_1_6gf
  4. regnet_y_3_2gf

MAXVIT:
  1. maxvit_t

Usage: model = create_model('model_name', pretrained=True)



In [3]:
TARGET_LABEL    = 'cecum'
MODEL_NAME      = 'convnext_small'
EXPERIMENT_NAME = f'{TARGET_LABEL}_{MODEL_NAME}'

BATCH_SIZE              = 16
NUM_WORKERS             = 32
IMG_SIZE                = 224
NUM_EPOCHS              = 500
LEARNING_RATE           = 0.001
WEIGHT_DECAY            = 0.0001
EARLY_STOPPING_PATIENCE = 20

NUM_CLASSES   = 2
PRETRAINED    = True
USE_MULTI_GPU = True

data_dir  = PROJECT_ROOT / "data" / "raw" / "CAS-Colon"
train_csv = data_dir / "Label" / "frame" / "Fold" / "train_fold_1.csv"
val_csv   = data_dir / "Label" / "frame" / "Fold" / "val_fold_1.csv"
image_dir = data_dir / "Frame"
save_dir  = PROJECT_ROOT / "outputs" / TARGET_LABEL


In [4]:
train_loader, val_loader, train_dataset, val_dataset = create_dataloaders(
    train_csv    = str(train_csv),
    val_csv      = str(val_csv),
    image_dir    = str(image_dir),
    target_label = TARGET_LABEL,
    batch_size   = BATCH_SIZE,
    num_workers  = NUM_WORKERS,
    img_size     = IMG_SIZE
)

In [5]:
# 모델 생성
model = create_model(
    model_name    = MODEL_NAME,
    pretrained    = PRETRAINED,
    device        = device,
    use_multi_gpu = USE_MULTI_GPU
)
print(f"Model: {MODEL_NAME}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")


Using 4 GPUs with DataParallel
Model: convnext_small
Total parameters: 49,456,226


In [ ]:
# 학습 (개선된 설정)
trainer = Trainer(
    model           = model,
    train_loader    = train_loader,
    val_loader      = val_loader,
    device          = device,
    save_dir        = str(save_dir),
    experiment_name = EXPERIMENT_NAME,
    use_amp         = True  # Mixed Precision Training
)

history = trainer.train(
    num_epochs              = NUM_EPOCHS,
    lr                      = LEARNING_RATE,
    weight_decay            = WEIGHT_DECAY,
    save_best               = True,
    early_stopping_patience = EARLY_STOPPING_PATIENCE,
    loss_type               = 'focal',  # 'focal', 'crossentropy', or 'label_smoothing'
    focal_gamma             = 2.0,      # Focal loss gamma parameter
    max_grad_norm           = 1.0,      # Gradient clipping
    scheduler_patience      = 5,        # LR scheduler patience
    scheduler_factor        = 0.5       # LR reduction factor
)

Using focal loss with class weights: [0.5307255776246453, 8.636543535620053]

Starting training: cecum_convnext_small
Loss Function: focal
Mixed Precision: True
Gradient Clipping: 1.0
Monitor: Validation F1 Score (higher is better)
Early Stopping Patience: 20



Epoch 1/500 | LR: 0.000976 | Train - Loss: 0.1959, F1: 0.1028 | Val - Loss: 0.1842, F1: 0.1236 | * Best F1: 0.1236


Training:  92%|█████████▏| 1508/1636 [59:35<05:04,  2.38s/it, loss=0.176] 

In [ ]:
plot_training_history(history, save_dir=str(save_dir), show=True)
print_training_summary(history)


In [ ]:
trainer.load_checkpoint(str(save_dir / "best_model.pth"))

evaluator = Evaluator(model, val_loader, device, str(save_dir / "evaluation"))
metrics, y_true, y_pred, y_probs = evaluator.evaluate(
    class_names=[f'Non-{TARGET_LABEL.capitalize()}', TARGET_LABEL.capitalize()],
    use_optimal_threshold=True
)